In [35]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import root_mean_squared_error

import mlflow

In [36]:
# mlflow ui --backend-store-uri sqlite://mlflow.db

# set the tracking uri
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# set experiment
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/deployment/mlruns/1', creation_time=1718315745854, experiment_id='1', last_update_time=1718315745854, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [37]:
def read_dataframe(filename):
  if filename.endswith('.csv'):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
  elif filename.endswith('.parquet'):
    df = pd.read_parquet(filename)

  df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
  df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

  df = df[(df.duration >= 1) & (df.duration <= 60)]

  categorical = ['PULocationID', 'DOLocationID']
  df[categorical] = df[categorical].astype(str)
    
  return df

In [38]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [39]:
len(df_train), len(df_val)

(73908, 61921)

In [40]:
def prepare_features(df, dv=None):
  # merge location columns
  df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

  # define predictor and target variables
  categorical = ['PU_DO'] #'PULocationID', 'DOLocationID'
  numerical = ['trip_distance']
  target = 'duration'

  # dataframe to dictionaries for vectorizing
  data_dicts = df[categorical + numerical].to_dict(orient='records')

  if dv:
    X = dv.transform(data_dicts)
  else:
    dv = DictVectorizer()
    X = dv.fit_transform(data_dicts)

  y = df[target].values

  return X, y, dv

In [41]:
X_train, y_train, dv = prepare_features(df=df_train)
X_val, y_val, _ = prepare_features(df=df_val, dv=dv)

In [42]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715200888857

In [ ]:
with mlflow.start_run():
  mlflow.set_tag('developer', 'Armand Winant')

  alpha = .01

  lr = Lasso(alpha)
  lr.fit()